# Deskripsi Proyek

Proyek ini bertujuan untuk membangun sebuah pipeline machine learning untuk memprediksi risiko hipertensi pada seseorang. Model akan dilatih berdasarkan berbagai fitur, termasuk data demografis, gaya hidup, dan riwayat kesehatan yang tersedia dalam dataset.

Pipeline ini dibangun menggunakan TensorFlow Extended (TFX) dan mencakup komponen-komponen standar sebagai berikut:
- Ingest data
- Validasi skema
- Transformasi fitur
- Pelatihan model
- Evaluasi model
- Pushing model akhir


# Import Library
Merupakan tahapan paling awal yang berguna untuk melakukan import library yang akan digunakan

In [1]:
import os
import tensorflow as tf
import tensorflow_model_analysis as tfma 
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.dsl.components.common.resolver import Resolver 
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 

# Set Variable
Merupakan setting awal untuk menetapkan beberapa variabel global

In [2]:
PIPELINE_NAME = "pipeline"
SCHEMA_PIPELINE_NAME = "schema"
PIPELINE_ROOT = 'naufal_azmi_ginting-pipeline' #os.path.join('naufal_azmi_ginting-pipeline', PIPELINE_NAME)
METADATA_PATH = os.path.join(PIPELINE_ROOT, 'metadata', 'metadata.db')
SERVING_MODEL_DIR = os.path.join(PIPELINE_ROOT, 'serving_model')
DATA_ROOT = "data"

In [3]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# ExampleGen
Membaca dataset dari file CSV dan mengonversinya menjadi format TFRecord.

In [4]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [5]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

# StatisticGen
Menghasilkan statistik deskriptif dari data untuk keperluan validasi dan deteksi anomali.

In [6]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [7]:
interactive_context.show(statistics_gen.outputs["statistics"])

# SchemaGen
Menghasilkan skema data berdasarkan statistik dari data training.

In [8]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [9]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Age',INT,required,,-
'BMI',FLOAT,required,,-
'BP_History',STRING,required,,'BP_History'
'Exercise_Level',STRING,required,,'Exercise_Level'
'Family_History',STRING,required,,'Family_History'
'Has_Hypertension',STRING,required,,'Has_Hypertension'
'Medication',STRING,required,,'Medication'
'Salt_Intake',FLOAT,required,,-
'Sleep_Duration',FLOAT,required,,-


,Values
Domain,
'BP_History',"'Hypertension', 'Normal', 'Prehypertension'"
'Exercise_Level',"'High', 'Low', 'Moderate'"
'Family_History',"'No', 'Yes'"
'Has_Hypertension',"'No', 'Yes'"
'Medication',"'ACE Inhibitor', 'Beta Blocker', 'Diuretic', 'None', 'Other'"
'Smoking_Status',"'Non-Smoker', 'Smoker'"


# ExampleValidator
Mengevaluasi validitas data dan mendeteksi nilai outlier atau missing berdasarkan skema.

In [10]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [11]:
interactive_context.show(example_validator.outputs['anomalies'])

# Transform
Merupakan tahapan preprocessing dataset agar data dapat diolah oleh model

In [12]:
TRANSFORM_MODULE_FILE = "modules\hypertension_transform.py"

In [13]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: naufal_azmi_ginting-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\0e9e5a93d87f43f793f542f082ea983c\assets


INFO:tensorflow:Assets written to: naufal_azmi_ginting-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\0e9e5a93d87f43f793f542f082ea983c\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

# Trainer
Model neural network sederhana dibangun menggunakan Keras Sequential. Model ini dilatih menggunakan data yang sudah ditransformasi sebelumnya.

Proses pelatihan menggunakan Trainer yang menyimpan model lengkap beserta preprocessing layer, sehingga saat serving nanti input tetap diproses sesuai transformasi training.

In [14]:
MODEL_FILE = "modules\hypertension_trainer.py"

In [15]:
trainer = Trainer(
    module_file=MODEL_FILE,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(num_steps=1000),
)
interactive_context.run(trainer)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 BP_History_xf (InputLayer)     [(None, 5)]          0           []                               
                                                                                                  
 Medication_xf (InputLayer)     [(None, 6)]          0           []                               
                                                                                                  
 Family_History_xf (InputLayer)  [(None, 3)]         0           []                               
                                                                                                  
 Exercise_Level_xf (InputLayer)  [(None, 4)]         0           []                               
                                                                                              

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: naufal_azmi_ginting-pipeline\Trainer\model\6\Format-Serving\assets


INFO:tensorflow:Assets written to: naufal_azmi_ginting-pipeline\Trainer\model\6\Format-Serving\assets


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

# Resolver
Untuk mengambil model terbaik dan terbaru dari hasil pipeline sebelumnya

In [16]:
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

# Evaluator
Komponen Evaluator mengevaluasi performa model pada data validasi. Evaluasi dilakukan berdasarkan metrik akurasi dan metrik lain seperti false positive atau false negative.

In [17]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='Has_Hypertension_xf')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

In [18]:
from pathlib import Path

transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri
assert Path(transform_graph_uri).exists(), "Transform graph path tidak ditemukan."

evaluator = Evaluator(
    examples=transform.outputs['transformed_examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config
)
 
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [19]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

# Pusher
Model dengan performa baik dipilih dan dikirim ke direktori serving. Model ini telah siap digunakan dalam aplikasi produksi

In [20]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR)))
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 9
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))